<a href="https://colab.research.google.com/github/yansigit/colab-ipynbs/blob/main/staitz_team_details_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Selenium

!apt-get update # to update ubuntu to correctly run apt install

!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
# re 0 -> 타자, 1 -> 투수
url = "http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re={}&ys={}&ye={}&se=0&te={}&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn="
teams = ['두산', '삼성', '키움', '한화', '롯데', 'NC', 'SK', 'LG', 'KIA', 'kt']
driver.implicitly_wait(5)

In [ ]:
import pandas as pd
import os

for team in teams:
    url = url.format(0, 2020, 2020, team)
    driver.get(url)
    a_players = driver.find_elements_by_xpath('//*[@id="fixcol"]/table/tbody/tr/td/a')

    urls_to_crawl = {}
    for a in a_players:
        name = a.get_attribute("innerText")
        urls_to_crawl[name] = (a.get_attribute('href') + "&opt=6")
        # 상대별 = opt5 playlog = opt6

    # df_list = []
    for name, _url in urls_to_crawl.items():
        driver.get(_url)
        table = driver.find_element_by_css_selector("table.table.table-striped.table-responsive")
        table_html = table.get_attribute("outerHTML")
        # df_list.append(pd.read_html(table_html)[0])
        df = pd.read_html(table_html)[0]
        os.makedirs("csv/" + team, exist_ok=True)
        df.to_csv("csv/{}/타자_{}_playlog.csv".format(team, name), encoding="utf-8-sig")
